In [1]:
# from py_pdf_parser.loaders import load_file
# # from py_pdf_parser.visualise import visualise

import pdfplumber


In [ ]:
import fitz
import pdfplumber
from transformers import pipeline

# 初始化翻译模型
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-zh")

# 1. 提取 PDF 内容
def extract_content(pdf_path):
    content = []
    doc = fitz.open(pdf_path)
    for page_number, page in enumerate(doc, start=1):
        for block in page.get_text("dict")["blocks"]:
            for line in block["lines"]:
                for span in line["spans"]:
                    text = span["text"]
                    bbox = span["bbox"]
                    font_size = span["size"]
                    content.append({"text": text, "bbox": bbox, "font_size": font_size, "page": page_number})
    return content

# 2. 翻译内容
def translate_content(content):
    for item in content:
        item["translated_text"] = translator(item["text"], max_length=400)[0]["translation_text"]
    return content

# 3. 生成新 PDF
def generate_pdf(content, original_pdf, output_pdf):
    doc = fitz.open(original_pdf)
    new_pdf = fitz.open()
    for page_number, page in enumerate(doc, start=1):
        new_page = new_pdf.new_page(width=page.rect.width, height=page.rect.height)
        new_page.show_pdf_page(page.rect, doc, page.number)
        for item in content:
            if item["page"] == page_number:
                new_page.insert_text(item["bbox"][:2], item["translated_text"], fontsize=item["font_size"])
    new_pdf.save(output_pdf)

# 主函数调用
pdf_path = "example.pdf"
content = extract_content(pdf_path)
content = translate_content(content)
generate_pdf(content, pdf_path, "translated_output.pdf")
